## 收集数据
- tweet无法访问 :)
- 从github仓库[https://github.com/udacity/new-dand-advanced-china](https://github.com/udacity/new-dand-advanced-china)中下载文件

In [2]:
# -*- coding=utf-8 -*-
import requests

def download(url):
    with open(url.split('/')[-1], mode="wb") as f:
        respone = requests.get(url)
        f.write(respone.content)
        
# 下载 image-predictions.tsv
image_prediction_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/image-predictions.tsv"
download(image_prediction_url)
# 下载 twitter_archive_enhanced.csv
twitter_archive_enhanced_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/twitter-archive-enhanced.csv"
download(twitter_archive_enhanced_url)
# 下载 tweet_json.txt
tweet_json_url = "https://raw.githubusercontent.com/udacity/new-dand-advanced-china/master/%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97/WeRateDogs%E9%A1%B9%E7%9B%AE/tweet_json.txt"
download(tweet_json_url)

## 评估数据

In [3]:
# 使用pandas读取数据文件
import pandas as pd

In [4]:
image_predictions = pd.read_csv("image-predictions.tsv", sep='\t')
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.0+ KB


In [5]:
image_predictions.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [8]:
# 处理和评估tweet_json.txt数据
import json
with open("tweet_json.txt", 'r') as json_file:
    for data in json_file.readlines():
        print(json.dumps(json.loads(data), indent=4, sort_keys=False, ensure_ascii=True))
        break

{
    "contributors": null, 
    "truncated": false, 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "id": 892420643555336193, 
    "favorite_count": 39492, 
    "full_text": "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU", 
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", 
    "retweeted": false, 
    "coordinates": null, 
    "entities": {
        "symbols": [], 
        "user_mentions": [], 
        "hashtags": [], 
        "urls": [], 
        "media": [
            {
                "expanded_url": "https://twitter.com/dog_rates/status/892420643555336193/photo/1", 
                "display_url": "pic.twitter.com/MgUWQ76dJU", 
                "url": "https://t.co/MgUWQ76dJU", 
                "media_url_https": "https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg", 
                "id_str": "892420639486877696", 
                "sizes": {
    

In [40]:
# 逐行读取tweet_json.txt文件并添加到pandas DataFrame中，（至少）包含 tweet ID、retweet_count和favorite_count字段...
# 提出问题，放到list中，然后通过 'id_str': list1, 'fav_count': list2的方式创建df的效率是否更高？
df_tweet = pd.DataFrame()

# 观察tweet_json.txt后，对以下数据感兴趣：
# retweet_count favorite_count full_text retweeted source favorited
with open("tweet_json.txt", 'r') as json_file:
    index = ['id', 'retweet_count', 'favorite_count', 'full_text', 'retweeted', 'source', 'favorited']
    for data in json_file.readlines():
        dict_data = json.loads(data)
        
        data = []
        for idx in index:
            data.append(dict_data[idx])
        
        s_tweet = pd.Series(data, index=index)
        retweet_count = dict_data[u'retweet_count']
        df_tweet = df_tweet.append(s_tweet, ignore_index=True)

df_tweet.head(3)

,favorite_count,favorited,full_text,id,retweet_count,retweeted,source
0,39492.0,0.0,This is Phineas. He's a mystical boy. Only eve...,8.924206e+17,8842.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
1,33786.0,0.0,This is Tilly. She's just checking pup on you....,8.921774e+17,6480.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."
2,25445.0,0.0,This is Archie. He is a rare Norwegian Pouncin...,8.918152e+17,4301.0,0.0,"<a href=""http://twitter.com/download/iphone"" r..."


In [41]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 7 columns):
favorite_count    2352 non-null float64
favorited         2352 non-null float64
full_text         2352 non-null object
id                2352 non-null float64
retweet_count     2352 non-null float64
retweeted         2352 non-null float64
source            2352 non-null object
dtypes: float64(5), object(2)
memory usage: 128.7+ KB


In [42]:
df_tweet.describe()

,favorite_count,favorited,id,retweet_count,retweeted
count,2352.000000,2352.0,2.352000e+03,2352.000000,2352.0
mean,8109.198980,0.0,7.425913e+17,3134.932398,0.0
std,11980.795669,0.0,6.846210e+16,5237.846296,0.0
min,0.000000,0.0,6.660209e+17,0.000000,0.0
25%,1417.000000,0.0,6.783949e+17,618.000000,0.0
50%,3596.500000,0.0,7.193536e+17,1456.500000,0.0
75%,10118.000000,0.0,7.991219e+17,3628.750000,0.0
max,132318.000000,0.0,8.924206e+17,79116.000000,0.0


In [28]:
df_tweet

,favorite_count,full_text,id,retweet_count,source
0,39492.0,This is Phineas. He's a mystical boy. Only eve...,8.924206e+17,8842.0,"<a href=""http://twitter.com/download/iphone"" r..."
1,33786.0,This is Tilly. She's just checking pup on you....,8.921774e+17,6480.0,"<a href=""http://twitter.com/download/iphone"" r..."
2,25445.0,This is Archie. He is a rare Norwegian Pouncin...,8.918152e+17,4301.0,"<a href=""http://twitter.com/download/iphone"" r..."
3,42863.0,This is Darla. She commenced a snooze mid meal...,8.916896e+17,8925.0,"<a href=""http://twitter.com/download/iphone"" r..."
4,41016.0,This is Franklin. He would like you to stop ca...,8.913276e+17,9721.0,"<a href=""http://twitter.com/download/iphone"" r..."
5,20548.0,Here we have a majestic great white breaching ...,8.910880e+17,3240.0,"<a href=""http://twitter.com/download/iphone"" r..."
6,12053.0,Meet Jax. He enjoys ice cream so much he gets ...,8.909719e+17,2142.0,"<a href=""http://twitter.com/download/iphone"" r..."
7,66596.0,When you watch your owner call another dog a g...,8.907292e+17,19548.0,"<a href=""http://twitter.com/download/iphone"" r..."
8,28187.0,This is Zoey. She doesn't want to be one of th...,8.906092e+17,4403.0,"<a href=""http://twitter.com/download/iphone"" r..."
9,32467.0,This is Cassie. She is a college pup. Studying...,8.902403e+17,7684.0,"<a href=""http://twitter.com/download/iphone"" r..."


In [44]:
witter_archive_enhanced = pd.read_csv("twitter-archive-enhanced.csv")
witter_archive_enhanced.info()
witter_archive_enhanced.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,NaN,NaN,NaN,NaN,10.000000,10.000000
50%,7.196279e+17,NaN,NaN,NaN,NaN,11.000000,10.000000
75%,7.993373e+17,NaN,NaN,NaN,NaN,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [45]:
witter_archive_enhanced

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


#### 质量
##### `df_tweet` 表格
- favorite_count、id、retweet_count应该是int64类型，而不是浮点浮点类型
- favorited 和 retweeted 的值都是零，需要删除这两列

##### `twitter_archive_enhanced` 表格
- doggo floofer pupper puppo字段有"None"这是python关键字，在csv体现为字符串，这应该在表示空
- in_reply_to_status_id和in_reply_to_user_id，只有78个值，这两个列的数据很可能没用
- timestamp、retweeted_status_timestamp应该是时间类型，现在是object类型
- rating_denominator不是10的需要清理掉, 比如170是最大值是不对的
- rating_numerator分子小于rating_denominator的需要清理掉
- expanded_urls的单条记录中有重复的url地址，清理清理只保留一个

#### 清洁度
##### `df_tweet` 表格
- source字段内容是html文本，需要更加简洁，只需要获取该html的text部分即可
- 通过witter_archive_enhanced.tweet_id和df_tweet.id和image_predictions.tweet_id关联创建新的表格

## 清理

In [46]:
# 清理前的备份
df_tweet_clean = df_tweet.copy()
witter_archive_enhanced_clean = witter_archive_enhanced.copy()

In [47]:
# favorite_count id retweet_count应该是int64类型，现在做格式转换 long
for i in ['favorite_count', 'id', 'retweet_count']:
    df_tweet_clean[i] = df_tweet_clean[i].astype(long)
df_tweet_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 7 columns):
favorite_count    2352 non-null int64
favorited         2352 non-null float64
full_text         2352 non-null object
id                2352 non-null int64
retweet_count     2352 non-null int64
retweeted         2352 non-null float64
source            2352 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 128.7+ KB


In [48]:
# 顺便调整列的顺序
index = ['id', 'retweet_count', 'favorite_count', 'full_text', 'source']
df_tweet_clean = df_tweet_clean.reindex_axis(index, axis=1)
df_tweet_clean

,id,retweet_count,favorite_count,full_text,source
0,892420643555336192,8842,39492,This is Phineas. He's a mystical boy. Only eve...,"<a href=""http://twitter.com/download/iphone"" r..."
1,892177421306343424,6480,33786,This is Tilly. She's just checking pup on you....,"<a href=""http://twitter.com/download/iphone"" r..."
2,891815181378084864,4301,25445,This is Archie. He is a rare Norwegian Pouncin...,"<a href=""http://twitter.com/download/iphone"" r..."
3,891689557279858688,8925,42863,This is Darla. She commenced a snooze mid meal...,"<a href=""http://twitter.com/download/iphone"" r..."
4,891327558926688256,9721,41016,This is Franklin. He would like you to stop ca...,"<a href=""http://twitter.com/download/iphone"" r..."
5,891087950875897856,3240,20548,Here we have a majestic great white breaching ...,"<a href=""http://twitter.com/download/iphone"" r..."
6,890971913173991424,2142,12053,Meet Jax. He enjoys ice cream so much he gets ...,"<a href=""http://twitter.com/download/iphone"" r..."
7,890729181411237888,19548,66596,When you watch your owner call another dog a g...,"<a href=""http://twitter.com/download/iphone"" r..."
8,890609185150312448,4403,28187,This is Zoey. She doesn't want to be one of th...,"<a href=""http://twitter.com/download/iphone"" r..."
9,890240255349198848,7684,32467,This is Cassie. She is a college pup. Studying...,"<a href=""http://twitter.com/download/iphone"" r..."


In [49]:
# rating_denominator不是10的需要清理掉
witter_archive_enhanced_clean = witter_archive_enhanced[witter_archive_enhanced["rating_denominator"]==10]
witter_archive_enhanced_clean.info()
witter_archive_enhanced_clean

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2333 non-null int64
in_reply_to_status_id         73 non-null float64
in_reply_to_user_id           73 non-null float64
timestamp                     2333 non-null object
source                        2333 non-null object
text                          2333 non-null object
retweeted_status_id           180 non-null float64
retweeted_status_user_id      180 non-null float64
retweeted_status_timestamp    180 non-null object
expanded_urls                 2278 non-null object
rating_numerator              2333 non-null int64
rating_denominator            2333 non-null int64
name                          2333 non-null object
doggo                         2333 non-null object
floofer                       2333 non-null object
pupper                        2333 non-null object
puppo                         2333 non-null object
dtypes: float64(4), int64(3), ob

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [50]:
# rating_numerator分子小于rating_denominator的需要清理掉
witter_archive_enhanced_clean = witter_archive_enhanced_clean[witter_archive_enhanced_clean["rating_numerator"] > witter_archive_enhanced_clean["rating_denominator"]]
witter_archive_enhanced_clean.info()
witter_archive_enhanced_clean

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1438 entries, 0 to 2339
Data columns (total 17 columns):
tweet_id                      1438 non-null int64
in_reply_to_status_id         56 non-null float64
in_reply_to_user_id           56 non-null float64
timestamp                     1438 non-null object
source                        1438 non-null object
text                          1438 non-null object
retweeted_status_id           153 non-null float64
retweeted_status_user_id      153 non-null float64
retweeted_status_timestamp    153 non-null object
expanded_urls                 1398 non-null object
rating_numerator              1438 non-null int64
rating_denominator            1438 non-null int64
name                          1438 non-null object
doggo                         1438 non-null object
floofer                       1438 non-null object
pupper                        1438 non-null object
puppo                         1438 non-null object
dtypes: float64(4), int64(3), ob

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [56]:
# expanded_urls的单条记录中有重复的url地址，清理清理只保留一个
witter_archive_enhanced_clean['expanded_urls'] = witter_archive_enhanced_clean['expanded_urls'].str.split(',')[0]
witter_archive_enhanced_clean

ValueError: Length of values does not match length of index